# Topic extraction from the GEPRiS dataset and creation of an user-centric visualisation
Author: Tim Korjakow        
Summer term 2018      
Freie Universität Berlin     
Fachgebiet Human-Centered Computing

![Process graph](nlpflowchart.svg)

In [1]:
%load_ext autoreload
%autoreload 2
from sklearn.metrics import silhouette_samples
from sklearn.pipeline import Pipeline
import numpy as np
import json
from scipy.stats import entropy
import bz2
import csv
from bokeh.palettes import d3

from Preprocessing.preprocessing import Preprocessing
from Embedding.embedding import Embedding
from Topicextraction.topicextraction import TopicExtraction
from Clustering.clustering import Clustering
from Planereduction.planereduction import PlaneReduction
from Linearization.linearization import mapToSpaceSampling, computeClusterTopography
from Debug.debug import Debug
from sklearn.manifold import MDS
from scipy.spatial.distance import jensenshannon
from sklearn.preprocessing import LabelEncoder


# interactivity
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider, Dropdown, FloatSlider, fixed
import ipywidgets as widgets
from IPython.display import display, Javascript, HTML
import pickle


unable to import 'smart_open.gcs', disabling that module


In [2]:
#with bz2.open('../../../assets/data/train.csv.bz2', mode='rt') as f:
#    csvreader = csv.reader(f)
#    traindata = Preprocessing().fit_transform((row[1] for row in csvreader))
#    f.seek(0)
#    doc2author = {i:row[3:] for i, row in enumerate(csvreader)}

In [3]:
#print(list(doc2author.items())[:10])

In [4]:
models = {
#	'Doc2Vec': Embedding(method='Doc2Vec', model_path='../../../assets/models/doc2vec/doc2vec.joblib'),
#	'TfIdf': Embedding(method='TfIdf', dict_path='../../../assets/models/dict/dict.joblib', model_path='../../../assets/models/tfidf/tfidf.joblib'),
	'ATM': Embedding(method='ATM', dict_path='../models/dict/dict.joblib', model_path='../models/atm/atm.joblib')
}

print('Finished loading')

Finished loading


In [7]:
models["ATM"].selector.transform(1).shape

from sklearn.neighbors import NearestNeighbors

(261, 100)

In [6]:
for topic in models["ATM"].selector.model.show_topics(num_topics=10):
    #print('Label: ' + topic_labels[topic[0]])
    words = ''
    for word, prob in models["ATM"].selector.model.show_topic(topic[0]):
        words += word + ' '
    print('Words: ' + words)
    print()

Words: Lästigkeit Produktentwicklungsphasen microphone Schädigungsmodelle Konstruktionswerkstoffen Substratmaterialien Wiedergabesystems Festkörperkontakt bspw auralization 

Words: nicotine schizophren Routineversorgung sleep attentional Horizontalzellen Herzfrequenzvariabilität Fovea Frühgeburt CASK 

Words: Werkstück Leichtbau Aluminiumlegierungen Drehzahl Kaltgasspritzen Prozesssicherheit Einsatzbedingungen Sprachverständlichkeit Audioqualität Fließspannung 

Words: Stählen Superlegierungen Duktilität Korrosionsbeständigkeit tribologische Legierungselemente Ermüdungseigenschaften Fügen Werkstoffverhalten Schädigungsentwicklung 

Words: Podozyten EGFR spermatogenesis Thrombozyten Fibrose ncRNAs pancreatic myeloid aureus glomerulären 

Words: Nitrosoverbindungen Vergiftung Ionischen Ruthenium syntheses itch Base Iridium Colletotrichum Einlagerung 

Words: Treg kardiale Astrozyten Schlaf Spermatogenese renalen Präeklampsie myokardialen Plazenta Niereninsuffizienz 

Words: peripher act

In [7]:
{subject:len(docs) for subject, docs in models["ATM"].selector.model.author2doc.items()}

{'Messsysteme': 285,
 'Biomedizinische Systemtechnik': 3,
 'Physische Geographie': 382,
 'Forstwissenschaften': 231,
 'Entwicklungsbiologie': 342,
 'Städtebau/Stadtentwicklung, Raumplanung, Verkehrs- und Infrastrukturplanung, Landschaftsplanung': 142,
 'Pathologie': 415,
 'Dermatologie': 566,
 'Hämatologie, Onkologie, Transfusionsmedizin': 802,
 'Hals-Nasen-Ohrenheilkunde': 111,
 'Strukturbiologie': 234,
 'Optik, Quantenoptik und Physik der Atome, Moleküle und Plasmen': 1152,
 'Molekulare Neurowissenschaft und Neurogenetik': 680,
 'Ethnologie und Europäische Ethnologie': 492,
 'Allgemeine Theoretische Chemie': 407,
 'Thermodynamik und Kinetik von Werkstoffen': 215,
 'Experimentelle Physik der kondensierten Materie': 2468,
 'Mechanisches Verhalten von Konstruktionswerkstoffen': 110,
 'Pädagogische Sozial- und Organisationsforschung': 11,
 'Mechanik': 551,
 'Zelluläre Neurowissenschaft': 661,
 'Theoretische Physik der kondensierten Materie': 853,
 'Pflanzenwissenschaften': 2618,
 'Geodäs

In [12]:

hellinger = lambda x,y : np.linalg.norm(np.sqrt(x)-np.sqrt(y))/np.sqrt(2)
pipe = Pipeline([('Embedding', models['ATM']),
                 ('EmbeddingData', Debug()),
                 ('PlaneReduction', PlaneReduction(2, method='TSNE', perplexity=6, metric=jensenshannon))], verbose=True)

tfs_plane, labels = pipe.fit_transform(np.ones((10,10)))

#tfs_reduced = pipe.named_steps.TopicExtractionData.data
#print(tfs_reduced.shape)

# compute linearization
tfs_mapped = mapToSpaceSampling(tfs_plane)

# compute cluster topography
uncertainty = entropy(np.array(pipe.named_steps.EmbeddingData.data[0]).T + 0.000001)

print(len(models["ATM"].selector.model.doc2author.items()))
interpolated_topography = computeClusterTopography(tfs_mapped, uncertainty, 600, 600, 'cubic')
dump = {
            'project_data': [{'mappoint':mappoint, 'embpoint':embpoint, 'cluster':cluster, 'entropy': entropy, 'title':title, 'doc_count':doc_count} for mappoint, embpoint, cluster, entropy, title, doc_count in zip(
                tfs_mapped.tolist(),
                tfs_plane.tolist(),
                LabelEncoder().fit_transform([x[2] for x in models["ATM"].selector.model.doc2author.values()]).tolist(),
                uncertainty.tolist(),
                list(models["ATM"].selector.model.id2author.values()),
                [len(models["ATM"].selector.model.author2doc[author]) for author in models["ATM"].selector.model.id2author.values()]
            )],
            'cluster_data': {
                'cluster_colour': d3['Category20'][4]
            },
            'cluster_topography': np.flip(interpolated_topography.T, axis=0).flatten().tolist(),
            'topography_width': 600,
            'topography_height': 600
        }

[Pipeline] ......... (step 1 of 3) Processing Embedding, total=   0.0s
[Pipeline] ..... (step 2 of 3) Processing EmbeddingData, total=   0.0s
[Pipeline] .... (step 3 of 3) Processing PlaneReduction, total=   3.4s
86911


In [13]:
def draw_scatter(data, width=600, height=600, viz='linearized'):
    display(Javascript("""
        (function(element){
            require(['scatter'], function(scatter) {
                scatter(element.get(0), %s, %d, %d, %s);
            });
        })(element);
    """ % (json.dumps(data), width, height, json.dumps(viz))))

In [14]:
def visualize(dump=None):
    # viz dimensions
    width = 600
    height = 600

    display(HTML(filename="scatter.css.html"))
    display(Javascript("require.config({paths: {d3: 'https://d3js.org/d3.v5.min'}});"))
    display(Javascript(filename="scatter.js"))
    draw_scatter(dump, width, height)
    with open('./dump.json', 'w+') as f:
        json.dump(dump, f)

In [15]:
w = interactive(visualize, dump=fixed(dump))
display(w)


interactive(children=(Output(),), _dom_classes=('widget-interact',))